In [1]:
import pandas as pd
df = pd.read_csv("readmissions.csv")  # Adjust path/name
print(df.head())       # First 5 rows
print(df.shape)        # Rows, columns (e.g., 101766, 50)
print(df.columns)      # Full column list
print(df.dtypes)       # Types (int64, bool)
print(df.isnull().sum())  # Missing values per column

   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0                14                  41               0               11   
1                 2                  30               0               12   
2                 5                  66               0               22   
3                 3                  63               0                8   
4                 5                  40               0                6   

   number_outpatient  number_emergency  number_inpatient  number_diagnoses  \
0                  0                 0                 0                 6   
1                  0                 0                 1                 9   
2                  1                 0                 2                 9   
3                  0                 0                 0                 8   
4                  0                 0                 1                 9   

   race_Caucasian  race_AfricanAmerican  ...  citoglipton_No  insulin_No  

In [3]:
print(df.isnull().sum())  # Missing values per column

time_in_hospital              0
num_lab_procedures            0
num_procedures                0
num_medications               0
number_outpatient             0
                             ..
metformin-rosiglitazone_No    0
metformin-pioglitazone_No     0
change_No                     0
diabetesMed_Yes               0
readmitted                    0
Length: 65, dtype: int64


In [5]:
df = df.drop_duplicates()
print(f"Rows after removing duplicates: {len(df)}")

Rows after removing duplicates: 25000


In [7]:
print(df[['time_in_hospital', 'num_medications', 'num_lab_procedures']].describe())

       time_in_hospital  num_medications  num_lab_procedures
count      25000.000000     25000.000000         25000.00000
mean           4.395640        15.988440            42.96012
std            2.991165         8.107743            19.76881
min            1.000000         1.000000             1.00000
25%            2.000000        10.000000            31.00000
50%            4.000000        15.000000            44.00000
75%            6.000000        20.000000            57.00000
max           14.000000        81.000000           126.00000


In [9]:
# Cap num_medications at 50 (reasonable upper limit)
df = df[df['num_medications'] <= 50]

# Cap num_lab_procedures at 100 (typical hospital limit)
df = df[df['num_lab_procedures'] <= 100]

print(f"Rows after outlier removal: {len(df)}")

Rows after outlier removal: 24884


In [11]:
print(df.dtypes)

time_in_hospital              int64
num_lab_procedures            int64
num_procedures                int64
num_medications               int64
number_outpatient             int64
                              ...  
metformin-rosiglitazone_No     bool
metformin-pioglitazone_No      bool
change_No                      bool
diabetesMed_Yes                bool
readmitted                    int64
Length: 65, dtype: object


In [13]:
# Convert boolean columns to bool
bool_cols = [col for col in df.columns if col.startswith('race_') or col.endswith('_No') or col.endswith('_Yes')]
for col in bool_cols:
    df[col] = df[col].astype(bool)
print(df.dtypes)  # Verify

time_in_hospital              int64
num_lab_procedures            int64
num_procedures                int64
num_medications               int64
number_outpatient             int64
                              ...  
metformin-rosiglitazone_No     bool
metformin-pioglitazone_No      bool
change_No                      bool
diabetesMed_Yes                bool
readmitted                    int64
Length: 65, dtype: object


In [15]:
df.to_csv("cleaned_readmissions.csv", index=False)
print("Saved cleaned data to cleaned_readmissions.csv")

Saved cleaned data to cleaned_readmissions.csv


In [17]:
import sqlite3
import pandas as pd

# Load cleaned data
df = pd.read_csv("cleaned_readmissions.csv")

# Connect to SQLite
conn = sqlite3.connect("hospital.db")
df.to_sql("readmissions", conn, if_exists="replace", index=False)
print("Loaded data into SQLite database: hospital.db")

Loaded data into SQLite database: hospital.db


In [19]:
query = """
SELECT 
    CASE 
        WHEN "age_[70-80)" = 1 THEN '70-80'
        WHEN "age_[60-70)" = 1 THEN '60-70'
        WHEN "age_[50-60)" = 1 THEN '50-60'
        WHEN "age_[80-90)" = 1 THEN '80-90'
        WHEN "age_[40-50)" = 1 THEN '40-50'
        ELSE 'Other'
    END AS age_group,
    AVG(time_in_hospital) AS avg_stay
FROM readmissions
GROUP BY age_group
"""
result = pd.read_sql(query, conn)
print(result)

  age_group  avg_stay
0     40-50  4.126656
1     50-60  4.079432
2     60-70  4.329455
3     70-80  4.571725
4     80-90  4.787394
5     Other  3.991103


In [21]:
query = """
SELECT readmitted, AVG(num_medications) AS avg_medications
FROM readmissions
GROUP BY readmitted
"""
result = pd.read_sql(query, conn)
print(result)

   readmitted  avg_medications
0           0        15.350181
1           1        16.366634


In [23]:
query = """
SELECT 
    CASE 
        WHEN "medical_specialty_Cardiology" = 1 THEN 'Cardiology'
        WHEN "medical_specialty_Emergency/Trauma" = 1 THEN 'Emergency'
        WHEN "medical_specialty_Surgery-General" = 1 THEN 'Surgery'
        ELSE 'Other'
    END AS specialty,
    AVG(time_in_hospital) AS avg_stay
FROM readmissions
WHERE "age_[70-80)" = 1 OR "age_[80-90)" = 1
GROUP BY specialty
"""
result = pd.read_sql(query, conn)
print(result)

    specialty  avg_stay
0  Cardiology  3.815686
1   Emergency  4.646356
2       Other  4.707038


In [25]:
query = """
SELECT 
    SUM(CASE WHEN insulin_No = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS insulin_usage_pct,
    SUM(CASE WHEN metformin_No = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS metformin_usage_pct
FROM readmissions
WHERE readmitted = 1
"""
result = pd.read_sql(query, conn)
print(result)

   insulin_usage_pct  metformin_usage_pct
0          55.641544            18.081083


In [27]:
query = """
SELECT 
    CASE 
        WHEN "age_[70-80)" = 1 THEN '70-80'
        WHEN "age_[60-70)" = 1 THEN '60-70'
        WHEN "age_[50-60)" = 1 THEN '50-60'
        WHEN "age_[80-90)" = 1 THEN '80-90'
        WHEN "age_[40-50)" = 1 THEN '40-50'
        ELSE 'Other'
    END AS age_group,
    AVG(time_in_hospital) AS avg_stay
FROM readmissions
GROUP BY age_group
"""
result = pd.read_sql(query, conn)
result.to_csv("delays_by_age.csv", index=False)
print("Saved delays_by_age.csv")

Saved delays_by_age.csv


In [29]:
query = """
SELECT readmitted, AVG(num_medications) AS avg_medications
FROM readmissions
GROUP BY readmitted
"""
result = pd.read_sql(query, conn)
result.to_csv("meds_by_readmission.csv", index=False)
print("Saved meds_by_readmission.csv")

Saved meds_by_readmission.csv


In [31]:
import pandas as pd
df = pd.read_csv("cleaned_readmissions.csv")

In [33]:
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_Caucasian,race_AfricanAmerican,...,citoglipton_No,insulin_No,glyburide-metformin_No,glipizide-metformin_No,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_No,diabetesMed_Yes,readmitted
0,14,41,0,11,0,0,0,6,True,False,...,True,True,True,True,True,True,True,True,True,0
1,2,30,0,12,0,0,1,9,True,False,...,True,False,True,True,True,True,True,False,True,1
2,5,66,0,22,1,0,2,9,True,False,...,True,True,True,True,True,True,True,True,True,1
3,3,63,0,8,0,0,0,8,True,False,...,True,True,True,True,True,True,True,True,True,1
4,5,40,0,6,0,0,1,9,True,False,...,True,True,True,True,True,True,True,True,False,0


In [35]:
features = ['time_in_hospital', 'num_medications', 'num_lab_procedures', 'number_diagnoses',
            'age_[70-80)', 'age_[60-70)', 'race_Caucasian', 'gender_Female']
X = df[features]
y = df['readmitted']

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.56
